In [ ]:
import networkx as nx
import numpy as np
import pandas as pd 
import operator
import matplotlib.pyplot as plt
import random

In [ ]:
nx_graphs = []
agg_graph = nx.MultiGraph()
venue_df_list = []
for i in range(31):
    # with open(f'..\ATX_Networks\sim_day_{i+1}.gexf') as graph_file:
    nx_graphs.append(nx.read_gexf(f'..\ATX_Networks\sim_day_{i+1}.gexf'))
    venue_df_list.append(nx.to_pandas_edgelist(nx_graphs[i],source="Source",target="Target"))

concat_venue_df = pd.concat(venue_df_list)
# concat_venue_df.columns = ['source','target','venueid']
concat_venue_df.head()


In [ ]:
places_df = pd.read_csv("../RVF_ATX_PID_HZ_Places_Lookup.tsv",delimiter='\t')
merged_df = concat_venue_df.merge(places_df, on='venueid')
merged_df.head()

In [ ]:
#time_spent, hour, zip, visits, venueid
unique_venues = set(merged_df["venueid"])
unique_hours = set(np.arange(25))
max_time_spent = merged_df['time_spent'].max()
min_time_spent = merged_df['time_spent'].min()
max_visits = merged_df['visits'].max()
min_visits = merged_df['visits'].min()
time_spent_range = set(np.arange(min_time_spent, max_time_spent+1))
visit_range = set(np.arange(min_visits,max_visits+1))
print(f"The range of time spent is from {min_time_spent} to {max_time_spent}")


In [ ]:
source_set = set(merged_df['Source'].unique())
target_set = set(merged_df['Target'].unique())
node_set = source_set.union(target_set)

In [ ]:
#Generate negative samples

neg_df = pd.DataFrame(columns=["Source, Target, venueid, hour, time_spent, visits"])
count = 0
while len(neg_df) < len(merged_df):
    neg_edge = random.sample(node_set,k=2)
    if len(merged_df.loc[
    ((merged_df['Source']==neg_edge[0]) &
    ((merged_df['Target'])==neg_edge[1])) | 
    ((merged_df['Target']==neg_edge[0]) &
    ((merged_df['Source'])==neg_edge[1]))])==0:
        # print("new neg sample")
        sample_hour = random.sample(unique_hours,k=1)
        sample_time_spent = random.sample(time_spent_range,k=1)
        sample_venueid = random.sample(unique_venues, k=1)
        sample_visits = random.sample(visit_range,k=1)
        neg_df = neg_df.append({
            "Source":neg_edge[0],
            "Target":neg_edge[1],
            "venueid":sample_venueid,
            "hour":sample_hour,
            "time_spent":sample_time_spent,
            "visits":sample_visits
        }, ignore_index=True)
        count+=1
        if (count%100==0):
            print(f"{(count/len(merged_df['Source'])*100)}")
        if (count%10000==0):
            neg_df.to_csv(f"neg_samples_{count}.csv")



In [51]:
neg_df = pd.read_csv("neg_samples_10000.csv")
features_df_neg = neg_df.copy()[["Source","Target","venueid","hour","time_spent","visits"]]
features_df_neg["link"] = 0
features_df_neg = features_df_neg.astype({"hour":"int32","visits":"int32","time_spent":"int32"})
features_df_neg.head()

,Source,Target,venueid,hour,time_spent,visits,link
0,4ed23ffbfe4a8997f5172d7d2da88b7b75357e4c21af6f...,d8e4658adb2cd20d208eb6f089e4c482bf7ec7fa084ea9...,4b8ab9c1f964a520a67c32e3,3,2062,25,0
1,eaa72c4becae4d2ab3591148f4108b054c4e4131b462c6...,c4a8e52f2e668dce7bde2ba9bc8d25403304f22b5af3d8...,52d182ce498e318bf44252b9,0,582,28,0
2,166a77ec53a664b5843209ffae9096aa8c3c0d7c50b6be...,76f0a03ed49c23a4c355887bcc17d3804ee59ef86af438...,5df64969ee0c6b0008830f39,7,1150,6,0
3,4ee265480f8d034e23799a86ca77bbd4714184ed0bb762...,db8d399352dce3ce77907a63b0b46868d8fafb952c7512...,4c043267187ec928eabcb67b,2,2091,24,0
4,f25ea6a51502faa56c1cad5879d4731686acb9563da459...,6412ee7431487ab88234c16c2c871a942d325d86a6992b...,5bf342d2418686002c8d6005,5,1148,25,0


In [ ]:

features_df_pos = merged_df.copy()[["Source","Target","venueid","hour","time_spent","visits"]]
features_df_pos["link"] = 1
features_df_pos.head()

In [ ]:
features_df_pos = features_df_pos.merge(places_df[['level1cat','venueid']], on='venueid')

In [52]:
features_df_neg = features_df_neg.merge(places_df[['level1cat','venueid']], on='venueid')

In [ ]:

size = len(node_set)
node_encodings = np.arange(size)
node_list = list(node_set)



In [ ]:
node_encoding_df = pd.DataFrame({'Node':node_list,'Source_Encoding':node_encodings})
encoded_df = features_df_pos.merge(node_encoding_df, left_on='Source',right_on='Node')
node_encoding_df = pd.DataFrame({'Node':node_list,'Target_Encoding':node_encodings})
encoded_df = encoded_df.merge(node_encoding_df, left_on='Target',right_on='Node')
encoded_df_pos = encoded_df.drop(['Node_x','Node_y'],axis=1)


In [53]:
node_encoding_df = pd.DataFrame({'Node':node_list,'Source_Encoding':node_encodings})
encoded_df = features_df_neg.merge(node_encoding_df, left_on='Source',right_on='Node')
node_encoding_df = pd.DataFrame({'Node':node_list,'Target_Encoding':node_encodings})
encoded_df = encoded_df.merge(node_encoding_df, left_on='Target',right_on='Node')
encoded_df_neg = encoded_df.drop(['Node_x','Node_y'],axis=1)

In [55]:
encoded_df_pos.head()

,venueid,hour,time_spent,visits,link,level1cat,Source_Encoding,Target_Encoding,cat_Arts & Entertainment,cat_Colleges & Universities,cat_Events,cat_Food,cat_Nightlife Spots,cat_Outdoors & Recreation,cat_Professional & Other Places,cat_Residences,cat_Shops & Services,cat_Travel & Transport
0,440da2cbf964a52091301fe3,2,976,18,1,Travel & Transport,7171,13621,0,0,0,0,0,0,0,0,0,1
1,440da2cbf964a52091301fe3,3,217,5,1,Travel & Transport,27937,13621,0,0,0,0,0,0,0,0,0,1
2,440da2cbf964a52091301fe3,3,214,4,1,Travel & Transport,7946,13621,0,0,0,0,0,0,0,0,0,1
3,440da2cbf964a52091301fe3,4,526,9,1,Travel & Transport,17546,13621,0,0,0,0,0,0,0,0,0,1
4,440da2cbf964a52091301fe3,4,202,4,1,Travel & Transport,17546,13621,0,0,0,0,0,0,0,0,0,1


In [54]:
encoded_df_neg.head()

,Source,Target,venueid,hour,time_spent,visits,link,level1cat,Source_Encoding,Target_Encoding
0,4ed23ffbfe4a8997f5172d7d2da88b7b75357e4c21af6f...,d8e4658adb2cd20d208eb6f089e4c482bf7ec7fa084ea9...,4b8ab9c1f964a520a67c32e3,3,2062,25,0,Shops & Services,6776,27655
1,cf0b83f6cde68d475abb3d82ed316273ed0856e30ce1ef...,f9e5bde0ec17235ca5e921717b57bc0ede0344fe8760d3...,4b8ab9c1f964a520a67c32e3,23,3325,29,0,Shops & Services,372,4173
2,8d9640e9e4d1419330f6cc9a3f024a318d27baf6d7c9f3...,f9e5bde0ec17235ca5e921717b57bc0ede0344fe8760d3...,4d8e2eb2d265236a3b160417,17,2538,53,0,Shops & Services,13361,4173
3,8d15f93208b4dd514314d37d072fa2cf90da354a3c4aba...,f9e5bde0ec17235ca5e921717b57bc0ede0344fe8760d3...,4e69760e18a83989ec0c4d75,18,2262,50,0,Professional & Other Places,24301,4173
4,47989cfef7f68cce18fa32b9e7f6fd253fb0a09b777d24...,f9e5bde0ec17235ca5e921717b57bc0ede0344fe8760d3...,512b6c79e4b034d2bf81e144,7,2832,33,0,Professional & Other Places,12000,4173


In [ ]:
# merged_df['level1cat'].value_counts()
encoded_df_pos['level1cat'] = encoded_df_pos['level1cat'].astype('category')
one_hot = pd.get_dummies(encoded_df_pos.level1cat,prefix="cat")
encoded_df_pos = encoded_df_pos.join(one_hot)
encoded_df_pos = encoded_df_pos.drop(["Source","Target"],axis=1)
encoded_df_pos.columns

In [56]:
encoded_df_neg['level1cat'] = encoded_df_neg['level1cat'].astype('category')
one_hot = pd.get_dummies(encoded_df_neg.level1cat,prefix="cat")
encoded_df_neg = encoded_df_neg.join(one_hot)
encoded_df_neg = encoded_df_neg.drop(["Source","Target"],axis=1)
encoded_df_neg.columns

Index(['venueid', 'hour', 'time_spent', 'visits', 'link', 'level1cat',
       'Source_Encoding', 'Target_Encoding', 'cat_Arts & Entertainment',
       'cat_Colleges & Universities', 'cat_Events', 'cat_Food',
       'cat_Nightlife Spots', 'cat_Outdoors & Recreation',
       'cat_Professional & Other Places', 'cat_Residences',
       'cat_Shops & Services', 'cat_Travel & Transport'],
      dtype='object')

In [57]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from datetime import datetime
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
from scipy.stats import randint as sp_randint
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import uniform as sp_uniform
from sklearn.model_selection import cross_val_score
import seaborn as sns
import matplotlib.pyplot as plt

In [58]:
def results(model, X_train, X_valid,y_train, y_valid):
    soft_probs = model.predict_proba(X_valid)
    valid_auc = roc_auc_score(y_valid, soft_probs[:,1])
    print(valid_auc)
    train_preds = model.predict(X_train)
    acc = accuracy_score(y_train, train_preds)
    print(acc)
    preds = model.predict(X_valid)
    acc = accuracy_score(y_valid, preds)
    print(acc)
    print(classification_report(y_valid,preds))

In [71]:
encoded_df = encoded_df_pos.append(encoded_df_neg)
X = encoded_df.drop(['venueid','link','level1cat'],axis=1)
y = encoded_df['link']
X_train, X_valid, y_train, y_valid = train_test_split(X,y,train_size = 0.75, test_size = 0.25,random_state = 42, shuffle = True, stratify=y)

In [74]:
from catboost import CatBoostClassifier, Pool
X.head()

,hour,time_spent,visits,Source_Encoding,Target_Encoding,cat_Arts & Entertainment,cat_Colleges & Universities,cat_Events,cat_Food,cat_Nightlife Spots,cat_Outdoors & Recreation,cat_Professional & Other Places,cat_Residences,cat_Shops & Services,cat_Travel & Transport
0,2,976,18,7171,13621,0,0,0,0,0,0,0,0,0,1
1,3,217,5,27937,13621,0,0,0,0,0,0,0,0,0,1
2,3,214,4,7946,13621,0,0,0,0,0,0,0,0,0,1
3,4,526,9,17546,13621,0,0,0,0,0,0,0,0,0,1
4,4,202,4,17546,13621,0,0,0,0,0,0,0,0,0,1


In [75]:
cat_indices = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
model_cat = CatBoostClassifier(cat_features=cat_indices)
model_cat.fit(X_train,y_train)
results(model_cat,X_train, X_valid, y_train, y_valid)